In [1]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import time
inicio = time.time()

In [2]:
def extract_data():
    raiz = Path.cwd() #capturo diretório atual
    pasta_dados = Path.joinpath(raiz,"dados") #defino onde estão os problemas
    prob_por_arquivo = 200 #informando quantos problemas por arquivo eu tenho
    conjuntos = [10,20,50,100,200,500,1000]

    for conjunto in conjuntos:  #circulo nos arquivos
        dados[conjunto] = {}
        arquivo = f"sch"+str(conjunto)+'.txt'
        print("Iniciando problema {}".format(arquivo))
        caminho_problema = Path.joinpath(pasta_dados,arquivo)
        df = pd.read_fwf(caminho_problema) #importo no df
        df.columns=["n","p","a","b"]
        df.drop(columns=["n"], inplace=True) #jogo fora columa inútil
        n=df.iloc[0,0] #guardo o tamanho de itens do problema
        print("n = ", n)  
        df.dropna(inplace=True) #jogo fora linhas inúteis
        for problema_numero in range(prob_por_arquivo): #vou circular nos problemas dentro do arquivo. Cada iteração deste for é um problema.
            prob = df[n*problema_numero:n*(problema_numero+1)]
            dados[conjunto][problema_numero+1] = {
                'pi': prob["p"].to_numpy(),
                'ai': prob["a"].to_numpy(),
                'bi': prob["b"].to_numpy()
            }
            ## nesse ponto tenho o parâmetros do problema atual guardadas nas variáveis pi, ai e bi, prontos para serem utilizados na modelagem.
    return dados

In [3]:
dados = {}
objetivos = {}
variaveis = {}
tempos = {}
dados = extract_data()
lista_hs = [0.8, 0.6, 0.4, 0.2]

h=0.8
conjunto = 1000
problema = 2
z_corte = 2

pi = []
ai = []
bi = []

pi = np.array(dados[conjunto][problema]['pi'])
ai = np.array(dados[conjunto][problema]['ai'])
bi = np.array(dados[conjunto][problema]['bi'])

d=sum(pi)*h

i_itens = len(pi)
i_index=list(range(i_itens))
i_np = np.array(i_index)

Iniciando problema sch10.txt
n =  10
Iniciando problema sch20.txt
n =  20
Iniciando problema sch50.txt
n =  50
Iniciando problema sch100.txt
n =  100
Iniciando problema sch200.txt
n =  200
Iniciando problema sch500.txt
n =  500
Iniciando problema sch1000.txt
n =  1000


In [4]:
sa = np.tile([True,False],int(i_itens*0.5)) #solucao atual

In [5]:
sa
 

array([ True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,

In [6]:
def transforma(solucao_partida):
    set_E = np.nonzero(solucao_partida == True)[0] #[0] para retornar o np array e não o tuple.
    set_T = np.nonzero(solucao_partida == False)[0]
    return set_E,set_T

def busca_total(solucao_partida): 
    objetivos = np.array([])
    sol_testada = np.array([],dtype=bool)
    for cada_serv in range(len(solucao_partida)):
        sol_em_teste = solucao_partida
        sol_em_teste[cada_serv] = not(solucao_partida[cada_serv]) #inverte o serviço de bolso.
        set_E = np.nonzero(solucao_partida == True)[0] #[0] para retornar o np array e não o tuple.
        set_T = np.nonzero(solucao_partida == False)[0]
        objetivos = np.append(objetivos,calcula_objetivo(set_E,set_T))
        sol_testada = np.append(sol_testada,sol_em_teste)
    n = len(objetivos)
    sol_testada = np.reshape(sol_testada,[n,n])
    return objetivos,sol_testada


def calcula_objetivo(set_E,set_T):
    ai_pi = ai[set_E]/pi[set_E] #apenas do set_E
    bi_pi = bi[set_T]/pi[set_T] #apenas do set_T
    set_E_pi = pi[set_E]
    set_T_pi = pi[set_T]
    ai_pi_decr = np.flip(np.lexsort((set_E_pi,ai_pi))) #ordem de ai_pi e depois por pi.
    bi_pi_decr = np.flip(np.lexsort((set_T_pi,bi_pi)))
    set_E = set_E[ai_pi_decr]
    set_T = set_T[bi_pi_decr]
    
    
    sum_pi = 0
    objetivo = 0
    for tarefa in set_E:
        objetivo += sum_pi * ai[tarefa]
        sum_pi += pi[tarefa]
    
    sum_pi = 0
    for tarefa in set_T:
        objetivo += (sum_pi+pi[tarefa])*bi[tarefa]
        sum_pi += pi[tarefa]

    if sum(pi[set_E])>d:
        print("Preencha aqui o retorno esperado por variante infactivel")
        return objetivo#,False #só estou prevendo retornar o objetivo.
        #poderia verificar factibilidade.
    
    return objetivo#,True
                           
 

In [7]:
%time objs,sols = busca_total(sa)

Wall time: 3.02 s


In [8]:
np.average(objs)

9429422.253

In [9]:
sa

array([False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True,

15476.0

array([18,  6, 19, 12, 18,  5, 16,  6,  7,  8, 18,  5, 19,  2,  8, 18, 12,
       11,  4, 16,  8, 19,  8, 16,  5, 16,  8, 18,  7, 18, 19,  6, 12, 18,
       18,  7,  6, 20, 16, 11, 19,  2,  7, 12, 19, 20, 10, 19, 16, 16,  8,
       18,  2, 19, 19, 20, 20, 16, 10, 12,  2,  5, 18, 18,  8, 12,  4, 16,
        5, 18, 10, 16, 20,  4,  6,  4,  4, 20,  8, 16, 19,  4,  8,  6, 16,
        2,  7,  7,  7, 10,  2,  7,  7,  7,  8, 18,  7, 12, 20,  8,  7, 16,
       18,  8, 18, 19,  5, 18,  4, 18,  8,  8,  7, 20,  7, 18, 19, 11,  8,
       12, 19, 16, 18,  2, 10, 16,  5, 12, 10, 19,  8, 16,  5, 18, 16, 11,
       10, 19, 18, 12,  8,  8,  7,  2,  7, 10, 18,  2, 18,  4,  2, 12, 19,
       18, 16,  4, 16, 11,  7, 12, 16, 18,  8, 18, 19,  8,  7, 10,  8, 10,
       10,  2,  6,  7,  7,  7, 11, 20,  2,  7, 18,  4, 18,  7, 18, 12,  8,
        6, 12,  4, 20, 11,  4,  6, 18, 16,  7, 20,  7, 20,  4, 18, 16, 16,
        6,  7, 20,  8,  8,  8,  5, 16,  7,  8,  7, 10,  8, 16,  7, 16, 11,
        2,  6, 16,  2, 19